### Notebook version of main.py

In [72]:
import numpy as np
from os.path import join as oj
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
import tensorflow.keras as keras
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


def FCNN():
    """Defines a fully connected network with two hidden layers of 100 and 100 neurons respectively with dropout"""
    model = Sequential()
    
    # add input layer
    model.add(Dense(units=300, input_dim=324, activation='relu'))
    
    # add dropout
    model.add(Dropout(0.5))
    
    # add a hidden layer
    model.add(Dense(units=100, input_dim=300, activation='relu'))
        
    # add the output layer
    model.add(Dense(units=10, input_dim=100,activation='softmax'))
    
    return model
    

def run(data_dir, labels, feature_type):
    """Run a simple fully connected neural network on the feature set provided"""    
    train_data = np.load(oj(data_dir, '{}_train.npy'.format(feature_type)))
    x_train = train_data[:, :-1]
    y_train = train_data[:, -1]
    
    # one-hot encode the results
    y_train = keras.utils.to_categorical(y_train)
    
    test_data = np.load(oj(data_dir, '{}_test.npy'.format(feature_type)))
    x_test = test_data[:, :-1]
    y_test = test_data[:, -1]
    y_test = keras.utils.to_categorical(y_test)
    
    # get model
    model = FCNN()
    sgd = optimizers.SGD(lr=0.001, momentum=0.9)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
    
    model.fit(x_train, y_train, epochs=150, batch_size=128, validation_data=(x_test, y_test))

In [73]:
from scripts import load_cifar10, train_vgg16, fcnn_test

# train and test dir will be saved in DATA_DIR - if already there then it will not reload
DATA_DIR = '/opt/data/'

# key word arguments for model training
kwargs = dict(
    batch_size=128, epochs=250, learning_rate=0.1, momentum=0.9, image_shape=(32, 32, 3),
    lr_decay=1e-6, lr_drop=20, extra_aug=True, paper_layers=True, all_frozen=False, decay=True,
    allTrainable=False
)

LABELS = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
FEATURES = 'HOG'  # options: HOG, vgg16, or flatten


if __name__ == '__main__':
    # load CIFAR10 data
    # load_cifar10.run(save_dir=DATA_DIR)

    # train vgg16 model
    # train_vgg16.run(data_dir=DATA_DIR, **kwargs)

    # extract histogram of oriented gradient features
    # hog_features.run(DATA_DIR, LABELS)

    # extract flatten pixel features
    # flatten_features.run(DATA_DIR, LABELS)

    # run FCNN tests
    run(DATA_DIR, LABELS, FEATURES)
    

Train on 50000 samples, validate on 10000 samples
Epoch 1/150
50000/50000 [==============================] - 4s 73us/step - loss: 2.2897 - acc: 0.1225 - val_loss: 2.2505 - val_acc: 0.2171
Epoch 2/150
50000/50000 [==============================] - 3s 57us/step - loss: 2.2271 - acc: 0.1890 - val_loss: 2.1610 - val_acc: 0.2913
Epoch 3/150
50000/50000 [==============================] - 3s 50us/step - loss: 2.1318 - acc: 0.2309 - val_loss: 2.0428 - val_acc: 0.3318
Epoch 4/150
50000/50000 [==============================] - 2s 36us/step - loss: 2.0352 - acc: 0.2641 - val_loss: 1.9507 - val_acc: 0.3550
Epoch 5/150
50000/50000 [==============================] - 2s 36us/step - loss: 1.9656 - acc: 0.2914 - val_loss: 1.8880 - val_acc: 0.3773
Epoch 6/150
50000/50000 [==============================] - 3s 57us/step - loss: 1.9141 - acc: 0.3157 - val_loss: 1.8343 - val_acc: 0.3889
Epoch 7/150
50000/50000 [==============================] - 3s 55us/step - loss: 1.8647 - acc: 0.3367 - val_loss: 1.7832 - 

50000/50000 [==============================] - 3s 51us/step - loss: 1.3623 - acc: 0.5240 - val_loss: 1.3161 - val_acc: 0.5359
Epoch 60/150
50000/50000 [==============================] - 3s 58us/step - loss: 1.3558 - acc: 0.5254 - val_loss: 1.3139 - val_acc: 0.5387
Epoch 61/150
50000/50000 [==============================] - 3s 59us/step - loss: 1.3561 - acc: 0.5260 - val_loss: 1.3122 - val_acc: 0.5344
Epoch 62/150
50000/50000 [==============================] - 3s 54us/step - loss: 1.3540 - acc: 0.5296 - val_loss: 1.3079 - val_acc: 0.5405
Epoch 63/150
50000/50000 [==============================] - 2s 36us/step - loss: 1.3509 - acc: 0.5273 - val_loss: 1.3056 - val_acc: 0.5419
Epoch 64/150
50000/50000 [==============================] - 2s 36us/step - loss: 1.3455 - acc: 0.5291 - val_loss: 1.3036 - val_acc: 0.5411
Epoch 65/150
50000/50000 [==============================] - 2s 37us/step - loss: 1.3425 - acc: 0.5308 - val_loss: 1.3009 - val_acc: 0.5413
Epoch 66/150
50000/50000 [==============

Epoch 118/150
50000/50000 [==============================] - 2s 40us/step - loss: 1.2350 - acc: 0.5693 - val_loss: 1.2103 - val_acc: 0.5755
Epoch 119/150
50000/50000 [==============================] - 2s 34us/step - loss: 1.2354 - acc: 0.5692 - val_loss: 1.2085 - val_acc: 0.5746
Epoch 120/150
50000/50000 [==============================] - 2s 35us/step - loss: 1.2309 - acc: 0.5707 - val_loss: 1.2081 - val_acc: 0.5778
Epoch 121/150
50000/50000 [==============================] - 2s 34us/step - loss: 1.2313 - acc: 0.5697 - val_loss: 1.2063 - val_acc: 0.5760
Epoch 122/150
50000/50000 [==============================] - 2s 35us/step - loss: 1.2289 - acc: 0.5727 - val_loss: 1.2054 - val_acc: 0.5774
Epoch 123/150
50000/50000 [==============================] - 2s 42us/step - loss: 1.2274 - acc: 0.5744 - val_loss: 1.2044 - val_acc: 0.5762
Epoch 124/150
50000/50000 [==============================] - 3s 60us/step - loss: 1.2264 - acc: 0.5740 - val_loss: 1.2025 - val_acc: 0.5792
Epoch 125/150
50000/

In [ ]:
# if rerunning fresh - run this to install missing dependencies
!pip install imageio --user